# Model Training

### 1.1 Import Data and Required Packages

In [13]:
# Basic import 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Modelling 

# importing algorithms
from sklearn.linear_model import LinearRegression , Lasso ,Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Ensemble methods 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor

# Model Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Model Selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV



# Data Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, PolynomialFeatures

# model tuning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# Model Saving
from joblib import dump, load

# ignore warnings
import warnings
warnings.filterwarnings('ignore')


#### Import CSV Data as Data Frame


In [7]:
df = pd.read_csv('data/stud.csv')

In [8]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


### Preparing X and y variables

In [9]:
X = df.drop(columns = ['math_score'],axis = 1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [10]:
cat_features = X.select_dtypes(include = 'object').columns.tolist()
for feature in cat_features:
    print(f"Categories in {feature} variable    : " ,end = " ")
    print(df[feature].unique())

Categories in gender variable    :  ['female' 'male']
Categories in race_ethnicity variable    :  ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in parental_level_of_education variable    :  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in lunch variable    :  ['standard' 'free/reduced']
Categories in test_preparation_course variable    :  ['none' 'completed']


In [11]:
y = df['math_score']

In [14]:
## Create columns transform with 3 types of transformers

num_featurres = X.select_dtypes(include = ['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include = 'object').columns.tolist()

numaric_transform = StandardScaler()
categorical_transform = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numaric_transform, num_featurres),
        ('cat', categorical_transform, cat_features)
    ]
)

In [15]:
X = preprocessor.fit_transform(X)

In [16]:
X.shape

(1000, 19)

In [17]:
# separate the data into train and test sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)
X_train.shape,X_test.shape

((800, 19), (200, 19))

### Create and Evaluate Function to give the metrics after training model

In [18]:
# create the fucntion to evaluate the model 
def evaluate_model(true , pred):
    mae = mean_absolute_error(true, pred)
    mse = mean_squared_error(true, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, pred)
    
    return mae, mse, rmse, r2
    
 